In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [ ]:
root_folder = '../build/out/00_vols'
meshestxt = root_folder + '/plots/meshes.txt'

In [ ]:
run_names = [l[:-1] for l in open(meshestxt, "r").readlines()]

# Load Metrics

In [ ]:
# CELL-BASED TRIANGULATION METRICS
run_metrics = {}
run_orbitpoints = {}
for run_name in run_names:
    metrics_file     = os.path.join(root_folder, '{}metrics.csv'.format(run_name))
    orbitpoints_file = os.path.join(root_folder, '{}orbitpoints.csv'.format(run_name))
    
    # print(os.path.exists(metrics_file), os.path.exists(orbitpoints_file))
    run_metrics[run_name] = pd.read_csv(metrics_file)
    run_orbitpoints[run_name] = pd.read_csv(orbitpoints_file)

In [ ]:
# HEAT VALUES
run_data = {}
for run_name in run_names:
    try:
        csv_path = os.path.join(root_folder, 'plots/{}heatvals.csv'.format(run_name))
        
        origin_index  = pd.read_csv(csv_path, header=None, skiprows=1, nrows=1).values[0][0]
        orbit_indices = list(pd.read_csv(csv_path, header=None, skiprows=3, nrows=1).values[0])
        heat_values   = pd.read_csv(csv_path, skiprows=4)
        orbit_heat_vals = heat_values.iloc[orbit_indices]
        run_data[run_name] = orbit_heat_vals
    except:
        print("aha")

In [ ]:
def join_run_values(vdict, rnames):
    val_dict = copy.deepcopy(vdict)
    for run_name in rnames:
        _, cellSize, rerat, facetSize, lloyd, perturb, exude, nflips, _ = run_name.split('_')
        n_entries = val_dict[run_name].shape[0]
        lloyd   = bool(int(lloyd))
        perturb = bool(int(perturb))
        exude   = bool(int(exude))
        nflips  = int(nflips)
        cellSize = float(cellSize)
        facetSize = float(facetSize)

        n_orbitpoints = len(run_orbitpoints[run_name])
        max_shift     = run_orbitpoints[run_name][' moved_by'].max()

        val_dict[run_name]['cellSize'] = [cellSize] * n_entries
        val_dict[run_name]['rerat']    = [rerat]    * n_entries
        val_dict[run_name]['lloyd']    = [lloyd] * n_entries
        val_dict[run_name]['perturb']  = [perturb] * n_entries
        val_dict[run_name]['exude']    = [exude] * n_entries
        val_dict[run_name]['nflips']   = [nflips] * n_entries
        val_dict[run_name]['n_orbitpoints'] = [n_orbitpoints] * n_entries
        val_dict[run_name]['max_shift'] = [max_shift] * n_entries

    return pd.concat([val_dict[run_name] for run_name in run_names], ignore_index=True)

# CELL-BASED METRICS

In [ ]:
metric_df = join_run_values(run_metrics, run_names)
display(metric_df.groupby('cellSize').mean())

In [ ]:
avg_by_size = metric_df.groupby('cellSize').mean().sort_index()

fig = plt.figure(figsize=(15, 4))
axes = fig.subplots(nrows=1, ncols=3)
avg_by_size.plot(y='minangle', ax=axes[0])
avg_by_size.plot(y='amips',    ax=axes[1])
avg_by_size.plot(y='volume',   ax=axes[2])
plt.show()

# HEAT VALUES

In [ ]:
df = join_run_values(run_data, run_names)
df.groupby('cellSize')[['h_fem', ' h_dec']].mean().plot()
plt.title('Mean')
df.groupby('cellSize')[['h_fem', ' h_dec']].var().plot()
plt.title('Var')
plt.show()